## Pipeline demo - Knitro VS Optizelle

Here, we run knitro to optimize then restart from the local minimum found with optizelle just to compare number precision of these solvers.

In [1]:
import os
import os.path as osp
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
from bending_validation import suppress_stdout as so
from elastic_rods import EnergyType, InterleavingType

## Input parameters

In [2]:
rod_length = 0.3534025445286393
width = rod_length / 15 * 5
thickness = width / 5 * 0.35

In [3]:
# Sphere 1
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), 
                             (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),
                             (0.1111, 0.1865, 0.5316))
RIBBON_CS = [thickness, width]
MODEL_NAME = "sphere_1"
MODEL_PATH = osp.join(weaving_dir + 'normalized_objs/models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 20
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'normalized_objs/surface_models/{}.obj'.format(MODEL_NAME))

## Optimizer Parameters

In [4]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 10;
OPTS.beta = 1e-8
OPTS.niter = 100
OPTS.verboseNonPosDef = False

In [5]:
rw = 0.1
sw = 0.1

## Helper functions

### Initialize Woven Linkage

There are two different types of linkage: the `RodLinkage` and the `SurfaceAttractedLinkage`. The `SurfaceAttractedLinkage` has the additional surface attraction weight and hence need the `surface_path` as parameter.

In [6]:
def initialize_linkage(surface_path = INPUT_SURFACE_PATH, 
                       useCenterline = True, 
                       cross_section = RIBBON_CS, 
                       subdivision_res = SUBDIVISION_RESOLUTION, 
                       model_path = MODEL_PATH):
    l = elastic_rods.SurfaceAttractedLinkage(surface_path, 
                                             useCenterline, 
                                             model_path, 
                                             subdivision_res, 
                                             False, 
                                             InterleavingType.triaxialWeave)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    l.set_holdClosestPointsFixed(True);
    l.set_attraction_tgt_joint_weight(0.01);
    l.attraction_weight = 100;
    return l

### Compute Equilibrium

Similarly there are two different types of viewer depending on whether the surface is visualized.

In [7]:
def get_linkage_eqm(l, opt, cam_param = default_camera_parameters, target_surf = None):
    elastic_rods.compute_equilibrium(l, options = opt)
    if (target_surf is None):
        view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    else:
        view = linkage_vis.LinkageViewerWithSurface(l, target_surf, width=1024, height=640)
    view.setCameraParams(cam_param)
    return l, view

# Curved Ribbon Optimization

Initialize the linkage. Currently there are two types of design parameters: rest lengths and rest curvatures. The user can choose to activate or deactivate each one. 

In [8]:
with so(): curved_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH,
                                               useCenterline = True, 
                                               model_path = MODEL_PATH, 
                                               cross_section = RIBBON_CS,
                                               subdivision_res = SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewer(curved_linkage)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

## Stage 1

Callback function for updating the viewer during design parameter solve (Stage 1)

In [9]:
def curved_callback(prob, i):
    if (i % 20) != 0: return
    curved_linkage_view.update()

In [10]:
curved_dpo = elastic_rods.get_designParameter_optimizer(curved_linkage, rw, sw, callback=curved_callback)
curved_dpo.options.niter = 10000
curved_dpp = curved_dpo.get_problem()

Because how the weights work, this following line should not be run a second time.

In [11]:
with so(): curved_cr = curved_dpo.optimize()

### Equilibrium solve

In [12]:
with so(): elastic_rods.compute_equilibrium(curved_linkage, options = OPTS)
curved_linkage_view.update()

## Stage 2

### This is the stage that currently uses knitro.

`linkage_optimization` is defined in `python_bindings/linkage_optimization.cc`. The `LinkageOptimization.cc .hh` files are for the XShell (the naming is confusing but this is the unmerged stage. We will unify this after the new solver is in place.

In [13]:
import linkage_optimization

### Initialize the optimizer

In [14]:
OPTS.niter = 25
useCenterline = True
curved_optimizer = linkage_optimization.WeavingOptimization(curved_linkage, 
                                                            INPUT_SURFACE_PATH, 
                                                            useCenterline, 
                                                            equilibrium_options=OPTS, 
                                                            pinJoint=0, 
                                                            useFixedJoint=False)
curved_optimizer.set_target_joint_position(curved_save_tgt_joint_pos)
curved_linkage_view.update()
OPTS.niter = 200

In [15]:
curved_optimizer.rl_regularization_weight = 0.1
curved_optimizer.smoothing_weight = 10
curved_optimizer.beta = 500000.0
curved_optimizer.gamma = 1

In [16]:
algorithm = linkage_optimization.WeavingOptAlgorithm.NEWTON_CG
def curved_update_viewer():
    curved_linkage_view.update()

We need to run the stage 2 optimizer first with high surface attraction weight.

In [17]:
use_KNITRO = False
minRestLen = RIBBON_CS[1]*2.0
trust_region_radius = 1.0
tolerance = 1e-2
if use_KNITRO:
    itermax = 2000
    curved_optimizer.WeavingOptimize(algorithm, itermax, trust_region_radius, tolerance, curved_update_viewer, minRestLen)
else:
    itermax = 50
    intpoint_barrier_height = 0.7
    tolerance /= 4 # Optizelle needs a smaller tolerance for giving the same results
    curved_optimizer.WeavingOptimizeOptizelle(algorithm, 
                                              itermax, 
                                              trust_region_radius, 
                                              tolerance, 
                                              intpoint_barrier_height, 
                                              minRestLen)

Then we lower this weight and allow the closest point projections to change.

In [18]:
curved_optimizer.setLinkageAttractionWeight(1e-5)
curved_optimizer.set_holdClosestPointsFixed(False)

In [19]:
minRestLen = RIBBON_CS[1]*2.0
trust_region_radius = 1.0
tolerance = 1e-2
if use_KNITRO:
    itermax = 2000
    curved_optimizer.WeavingOptimize(algorithm, itermax, trust_region_radius, tolerance, curved_update_viewer, minRestLen)
else:
    itermax = 50
    intpoint_barrier_height = 0.7
    tolerance /= 10 # Optizelle needs a smaller tolerance for giving the same results
    curved_optimizer.WeavingOptimizeOptizelle(algorithm, 
                                              itermax, 
                                              trust_region_radius, 
                                              tolerance, 
                                              intpoint_barrier_height, 
                                              minRestLen)

### Validate whether the surface attraction weight is set low enough.

In [20]:
curved_optimizer_energy = curved_linkage.energy()
validation_curved_linkage = curved_optimizer.getLinesearchWeaverLinkage()
validation_curved_linkage.attraction_weight = 1e-7
with so(): elastic_rods.compute_equilibrium(validation_curved_linkage, options = OPTS)
validation_curved_view = linkage_vis.LinkageViewer(validation_curved_linkage, width=1024, height=640)
validation_curved_energy = validation_curved_linkage.energy()
print(abs((validation_curved_energy-curved_optimizer_energy)/curved_optimizer_energy))

0.000556150244432203
